In [ ]:
!pip install protobuf==3.20.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# import numpy as np

# data = np.load('/kaggle/input/preprocess-eyepacs/images_array.npz', mmap_mode='r')

# print(data.files)

# arr1 = data['arr_0']   # <-- this is now a memmap, NOT loaded into RAM
# print(type(arr1))
# print(arr1.shape)


In [ ]:
# arr1.shape

In [ ]:
# /kaggle/input/resnet50-trained-on-ptos19-eyepacs15-messidor-2

In [ ]:
# arr1[0].shape

In [ ]:
# plt.imshow(arr1[0])

In [ ]:
# images = np.read("/kaggle/input/preprocess-eyepacs/images_array.npz") 
# classes = pd.read_csv("/kaggle/input/preprocess-eyepacs/mycsvfile.csv")
# Y=classes["diagnosis"].shape

In [ ]:
# import numpy as np
# from tensorflow.keras.applications.resnet50 import preprocess_input

# batch_size = 256
# processed = []

# for i in range(0, arr1.shape[0], batch_size):
#     batch = arr1[i:i+batch_size]
#     batch = preprocess_input(batch)     # safe: only 256 images at once
#     processed.append(batch)

# X_processed = np.concatenate(processed, axis=0)


In [ ]:
# from tensorflow.keras.applications import ResNet50

# base_model = ResNet50(
#     weights="imagenet",
#     include_top=False,       # exclude the 1000-class head
#     input_shape=(224, 224, 3)
# )
# base_model.summary()

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model, layers

def build_backbone():
    base = ResNet50(include_top=False, weights='imagenet', input_shape=(None,None,3))

    # Extract outputs after Block 3
    output = base.get_layer("conv4_block6_out").output  # Block 3 output

    model = Model(inputs=base.input, outputs=output)
    return model


In [ ]:
def ASPP(inputs, out_channels=256, rates=(6,12,18)):
    # 1×1 branch
    x1 = layers.Conv2D(out_channels, 1, padding="same", use_bias=False)(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation("relu")(x1)

    branches = [x1]

    # Dilated conv branches
    for r in rates:
        x = layers.Conv2D(
            out_channels, 3,
            dilation_rate=r,
            padding="same",
            use_bias=False
        )(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        branches.append(x)

    # Global context branch
    x5 = layers.GlobalAveragePooling2D()(inputs)
    x5 = layers.Dense(out_channels, use_bias=False)(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation("relu")(x5)
    x5 = layers.Reshape((1,1,out_channels))(x5)

    # 🔑 Keras-safe dynamic resize
    x5 = layers.Lambda(
        lambda t: tf.image.resize(t[0], tf.shape(t[1])[1:3]),
        name="aspp_resize"
    )([x5, inputs])

    branches.append(x5)

    x = layers.Concatenate()(branches)
    x = layers.Conv2D(out_channels, 1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x



import tensorflow as tf
from tensorflow.keras import layers

def ASPP_v2(inputs, out_channels=256, rates=(6,12,18)):
    # 1x1 conv branch
    x1 = layers.Conv2D(out_channels, 1, padding="same", use_bias=False)(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation("relu")(x1)

    branches = [x1]

    # dilated conv branches
    for r in rates:
        x = layers.Conv2D(out_channels, 3, dilation_rate=r, padding="same", use_bias=False)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        branches.append(x)

    # global branch
    x5 = layers.GlobalAveragePooling2D()(inputs)            # (B, C_in)
    x5 = layers.Dense(out_channels, use_bias=False)(x5)    # (B, out_channels)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.Activation("relu")(x5)
    x5 = layers.Reshape((1,1,out_channels))(x5)
    # dynamic resize to (H, W)
    size = tf.shape(inputs)[1:3]
    x5 = tf.image.resize(x5, size, method='bilinear')

    branches.append(x5)

    x = layers.Concatenate()(branches)    # (B, H, W, out_channels * (len(rates)+2))
    x = layers.Conv2D(out_channels, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x


In [ ]:
def multi_task_classifier(features):
    x = layers.GlobalAveragePooling2D()(features)
    x = layers.Dense(512, activation="relu")(x)

    dr_output = layers.Dense(1, activation = "relu", name = "dr_output")(x)
    dme_output = layers.Dense(3, activation="softmax", name="dme_risk")(x)

    return dr_output, dme_output


In [ ]:
def build_model():
    inp = layers.Input((None,None,3))
    
    backbone = build_backbone()
    for layer in backbone.layers[:100]:
        layer.trainable = False
    features = backbone(inp)

    aspp = ASPP(features)
    dr, dme = multi_task_classifier(aspp)

    model = Model(inputs=inp, outputs=[dr, dme])
    return model

model = build_model()
model.summary()


In [ ]:
import tensorflow as tf
import numpy as np

class QuadraticKappa(tf.keras.metrics.Metric):
    def __init__(self, name="qwk", num_classes=5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.conf_mat = self.add_weight(
            name="conf_mat",
            shape=(num_classes, num_classes),
            initializer="zeros",
            dtype=tf.float32
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_true: (batch, 1) or (batch,)
        # y_pred: (batch, 1)

        y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)

        # Round + clip predictions
        y_pred = tf.clip_by_value(tf.round(y_pred), 0, self.num_classes - 1)
        y_pred = tf.cast(tf.reshape(y_pred, [-1]), tf.int32)

        cm = tf.math.confusion_matrix(
            y_true, y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )

        self.conf_mat.assign_add(cm)

    def result(self):
        num_classes = self.num_classes
        conf_mat = self.conf_mat

        # Weight matrix
        w = tf.zeros((num_classes, num_classes))
        for i in range(num_classes):
            for j in range(num_classes):
                w = tf.tensor_scatter_nd_update(
                    w, [[i, j]],
                    [((i - j) ** 2) / ((num_classes - 1) ** 2)]
                )

        act_hist = tf.reduce_sum(conf_mat, axis=1)
        pred_hist = tf.reduce_sum(conf_mat, axis=0)

        expected = tf.tensordot(act_hist, pred_hist, axes=0)
        expected = expected / tf.reduce_sum(expected)

        conf_norm = conf_mat / tf.reduce_sum(conf_mat)

        kappa = 1.0 - tf.reduce_sum(w * conf_norm) / tf.reduce_sum(w * expected)
        return kappa

    def reset_states(self):
        tf.keras.backend.set_value(self.conf_mat, np.zeros((self.num_classes, self.num_classes)))


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss={
        "dr_output": tf.keras.losses.Huber(delta=1.0),
        "dme_risk": "categorical_crossentropy"
    },
    loss_weights={
        "dr_output": 1.0,
        "dme_risk": 0.5
    },
    metrics={
        "dr_output": [QuadraticKappa(num_classes=5), "mae"],
        "dme_risk": ["accuracy"]
    }
)


In [ ]:
import numpy as np
import pandas as pd

npz = np.load(
    "/kaggle/input/preprocess-eyepacs/images_array.npz",
    mmap_mode="r"
)

images_preloaded = npz["arr_0"]   # memmap object, NOT in RAM
print(type(images_preloaded))     # <class 'numpy.memmap'>
print(images_preloaded.shape)


In [ ]:
classes = pd.read_csv("/kaggle/input/preprocess-eyepacs/mycsvfile.csv")

dr_labels = classes["diagnosis"].values.astype("float32")


In [ ]:
# def generator_no_dme(images, dr_labels, batch_size=16):
#     idx = np.arange(len(images))
#     while True:
#         np.random.shuffle(idx)
#         for i in range(0, len(idx), batch_size):
#             batch = idx[i:i+batch_size]

#             x = images[batch].astype("float32")
#             x = preprocess_input(x)

#             y = {
#                 "dr_output": dr_labels[batch],
#                 "dme_risk": np.zeros((len(batch), 2))  # dummy
#             }

#             sample_weight = {
#                 "dr_output": np.ones(len(batch)),
#                 "dme_risk": np.zeros(len(batch))  # 🚫 masked
#             }

#             yield x, y, sample_weight


In [ ]:
# model.fit(
#     generator_no_dme(images_A, dr_A),
#     steps_per_epoch=len(images_A)//16,
#     epochs=10
# )
# # 

In [ ]:
# def generator_with_dme(images, dr_labels, dme_labels, batch_size=16):
#     idx = np.arange(len(images))
#     while True:
#         np.random.shuffle(idx)
#         for i in range(0, len(idx), batch_size):
#             batch = idx[i:i+batch_size]
# # 
#             x = images[batch].astype("float32")
#             x = preprocess_input(x)

#             y = {
#                 "dr_output": dr_labels[batch],
#                 "dme_risk": dme_labels[batch]
#             }

#             sample_weight = {
#                 "dr_output": np.ones(len(batch)),
#                 "dme_risk": np.ones(len(batch))
#             }

#             yield x, y, sample_weight


In [ ]:
# model.fit(
#     generator_with_dme(images_B, dr_B, dme_B),
#     steps_per_epoch=len(images_B)//16,
#     epochs=30
# )


In [ ]:
class DROnlySequence(tf.keras.utils.Sequence):
    def __init__(self, images, dr_labels, batch_size=8, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.images = images
        self.dr_labels = dr_labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(images))
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        X = self.images[batch_idx].astype("float32")
        # X = preprocess_input(X)

        y_dr = self.dr_labels[batch_idx].astype("float32").reshape(-1, 1)

        return X, y_dr


In [ ]:
model_dr = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer("dr_output").output
)

model_dr.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="mse",     # later replace with ordinal/QWK-aware loss
    metrics=["mae"]
)

train_seq = DROnlySequence(
    images=images_preloaded,
    dr_labels=dr_labels,
    batch_size=8
)

model_dr.fit(train_seq, epochs=10)


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(
    model,
    to_file="model_graph.png",
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,   # VERY IMPORTANT (shows backbone + ASPP)
    dpi=200
)


In [ ]:
model.save_weights("pretrain_final.weights.h5")
